In [1]:
import pandas
import nipy
from nipy.modalities.fmri import hrf

In [2]:
df = pandas.read_csv('data/sim-data_drift-SP-variability-estim-onep.csv', index_col=0)

In [3]:
import numpy as np

In [4]:
df['onset'] = np.arange(10, len(df)*10 + 1, 10)

In [5]:
def get_bold_response(df, method='time_on_task',
                      sample_rate=1000):
    
    neural_signal = np.zeros((df.onset.max() + 20) * sample_rate)
    
    hrf = nipy.modalities.fmri.hrf.spm_hrf_compat(np.arange(0, 20, 1./sample_rate))
    
    for ix, row in df.iterrows():
        
        if method == 'time_on_task':
            start_idx = int(row.onset * sample_rate)
            end_idx = int((row.onset + row.rt)*sample_rate)
            value = 1
            
        elif method == 'time_on_task':
            start_idx = int(row.onset * sample_rate)
            end_idx = int((row.onset + row.rt)*sample_rate)
            value = 1            
        
        neural_signal[start_idx:end_idx] = value
    
    bold_signal = np.convolve(hrf, neural_signal)
    
    return neural_signal, bold_signal
    
    

# Time on task

In [ ]:


import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
sample_rate = 100

In [42]:
neural_signal, bold_signal = get_bold_response(df, sample_rate=sample_rate)

In [43]:
from nipy.modalities.fmri.experimental_paradigm import EventRelatedParadigm

In [44]:
paradigm = EventRelatedParadigm(['trial %d' % (ix+1) for ix in df.index], df.onset.tolist())

In [45]:
from scipy import signal

In [46]:
tr = 2.0

In [47]:
len(bold_signal) / sample_rate * tr

40079.98

In [48]:
bold_signal = signal.resample_poly(bold_signal, 1, sample_rate * tr)

In [49]:
import nipy.modalities.fmri.design_matrix as dm


In [87]:
n_scans = len(bold_signal)

# paradigm
frametimes = np.linspace(0, (n_scans - 1) * tr, n_scans)

# confounds
hrf_model = 'canonical'
drift_model = "polynomial"

X, names = dm.dmtx_light(frametimes, paradigm, drift_model=drift_model, drift_order=0, hrf_model=hrf_model)

In [88]:
import statsmodels.api as sm

In [89]:
model = sm.OLS(bold_signal, pandas.DataFrame(X, columns=names))

In [90]:
results = model.fit()

In [110]:
single_trial_betas = pandas.DataFrame(results.params).reset_index()
single_trial_betas = single_trial_betas.drop(2000)
single_trial_betas = single_trial_betas.rename(columns={0:'beta'})
single_trial_betas['trial'] = single_trial_betas['index'].apply(lambda x: int(x.split()[-1]))

In [115]:
df = df.merge(single_trial_betas, on='trial')

In [116]:
df.to_csv('data/sim-data_drift-SP-variability-estim-onep_time_on_task_beta.csv')

In [117]:
import seaborn as sns

In [ ]:
sns.lmplot('beta', 'rt')

# time_on_task * drift